# Experiment Data Collection
### Running the Experiments

The method ```create_network``` begins with a specified graph, and adds nodes one by one. using the function ```create_edge```.
With each node added, edges are also added according to specified rules (*binomial / scale-free / directed binomial / directed scale-free*). 

In [25]:
import networkx as nx
import random

# Setting seed.
random.seed(101)

# Binomial edge allocation probability.
binomial_probability = 0.2

# Setting recursion rules
recursion_level = 0
recursion_threshold = 100

def create_edge(G, i, j, capacity, changes, edge_reallocation_algorithm, directed):
    edge_reallocation = True
    G.add_edge(i, j)
    changes.append(('add', i, j)) # CHANGE = ADD
    if not directed:
        if G.degree(i) > capacity:
            edge_reallocation = reallocate_edges(G, i, capacity, 
                                                 changes, 
                                                 edge_reallocation_algorithm, 
                                                 directed)
        if G.degree(j) > capacity:
            edge_reallocation = reallocate_edges(G, j, capacity, 
                                                 changes, 
                                                 edge_reallocation_algorithm, 
                                                 directed)
    elif directed:
        if G.in_degree(j) > capacity:
            edge_reallocation = reallocate_edges(G, j, capacity, 
                                                 changes, 
                                                 edge_reallocation_algorithm, 
                                                 directed)
            
    if not edge_reallocation:
        return False
    return True

def reallocate_edges(G, node, capacity, changes, edge_reallocation_algorithm, directed):
    # Handle recursion.
    # global recursion_level
    # global recursion_threshold
    # if recursion_level > recursion_threshold:
    #     return False
    # recursion_level += 1 
    # if not directed:
    #     neighbors = list(G.neighbors(node))
    #     for neighbor in neighbors:
    #         G.remove_edge(node, neighbor)
    #         # Update changes.
    #         changes.append(('remove', node, neighbor)) # CHANGE = REMOVE
    #     for neighbor in neighbors: # Loop through each neighbor.
    #         # Potential nodes = list of nodes with no edge to the neighbor (that aren't the neighbor).
    #         potential_nodes = [n for n in G.nodes if not G.has_edge(node, n) and n != neighbor]
    #         if not potential_nodes:
    #             print(f"No potential nodes for node {node}")
    #             continue
    #         if edge_reallocation_algorithm == 'binomial': # If the *reallocation algorithm* is binomial.
    #             selected_node = random.choice(potential_nodes)
    #         elif edge_reallocation_algorithm == 'scale-free': # If the *reallocation algorithm* is scale-free.
    #             degrees = [G.degree(n) + 1 for n in potential_nodes]
    #             selected_node = random.choices(potential_nodes, weights=degrees, k=1)[0]
    #         create_edge(G, neighbor, selected_node, capacity, 
    #                     changes, 
    #                     edge_reallocation_algorithm, 
    #                     directed)
    # if directed:
    #     predecessors = list(G.predecessors(node))
    #     successors = list(G.successors(node))
    #     for predecessor in predecessors:
    #         # Remove edge: predecessor --> node
    #         G.remove_edge(predecessor, node)
    #         # Update changes.
    #         changes.append(('remove', predecessor, node)) # CHANGE = REMOVE
    #     for successor in successors:
    #         # Remove edge: node --> successor
    #         G.remove_edge(node, successor)
    #         # Update changes.
    #         changes.append(('remove', node, successor)) # CHANGE = REMOVE 
    #     for predecessor in predecessors:
    #         potential_nodes = [n for n in G.nodes if not G.has_edge(predecessor, n) and n != predecessor]
    #         if not potential_nodes:
    #             print(f"No potential nodes for node {node}")
    #             continue
    #         if edge_reallocation_algorithm == 'binomial':
    #             selected_node = random.choice(potential_nodes)
    #         elif edge_reallocation_algorithm == 'scale-free':
    #             degrees = [G.in_degree(n) + 1 for n in potential_nodes]
    #             selected_node = random.choices(potential_nodes, weights=degrees, k=1)[0]
    #         else:
    #             raise ValueError(f"Unexpected edge_reallocation_algorithm: {edge_reallocation_algorithm}")
    #         create_edge(G, predecessor, selected_node, 
    #                     capacity, changes, 
    #                     edge_reallocation_algorithm, 
    #                     directed)
    # recursion_level = 0
    return True

# Network Creation
def create_network(n_nodes, capacity, edge_allocation_algorithm, edge_reallocation_algorithm, directed):
    global binomial_probability
    changes = []
    if not directed:
        G_init = nx.Graph()
    elif directed:
        G_init = nx.DiGraph()
    else:
        raise ValueError('Directed is not boolean:', directed)
    G = G_init.copy()  
    for i in range(n_nodes):
        G.add_node(i)
        # If there's more than one node.
        if i > 0:
            # Go through each non-self node.
            for j in range(i - 1):
                if edge_allocation_algorithm == 'binomial':
                    probability = binomial_probability
                elif edge_allocation_algorithm == 'scale-free':
                    if not directed:
                        degree = G.degree(j)
                    else:
                        degree = G.in_degree(j)
                    # Weighted probabilities range between 0.1-0.5
                    print(probability)
                    probability = (degree + 0.1 * capacity) / (2 * capacity)
                if random.random() < probability:
                    edge_creation = create_edge(G, i, j, capacity, changes, edge_reallocation_algorithm, directed)
                    if not edge_creation:
                        print('Recursion will likely be infinite. Ending experiment.')
                        return G_init, changes
    return G_init, changes

### Writing the Changes to GEXF files.

The create_network function returns the initial graph and a list of changes made to it.
These are subsequently stored in a GEXF file.

In [10]:
def write_to_gexfs(G_init, changes, path):
    G = G_init.copy()  # Make a copy of the initial graph
    for timestep, change in enumerate(changes):
        operation, i, j = change
        if operation == 'add':
            G.add_edge(i, j)
        elif operation == 'remove':
            G.remove_edge(i, j)
        else:
            raise ValueError (timestep)
        nx.write_gexf(G, f"{path}/timestep_{timestep}.gexf") # type: ignore

In [19]:
import os

def run_experiment(n, capacity, parameters):
    # edge_allocation_algorithm, edge_reallocation_algorithm, directed, name
    edge_allocation_algorithm = parameters[0]
    edge_reallocation_algorithm = parameters[1]
    directed = True if parameters[2] == 'directed' else False
    name = parameters[3]
    print(f'Experiment, {name}')
    G_init, changes = create_network(n, capacity, edge_allocation_algorithm, edge_reallocation_algorithm, directed)
    path = f'/Users/aidanlowrie/Documents/GitHub/MRWPProject/Aidan_stuff/experiment_data/{name}'
    os.makedirs(path, exist_ok=True)
    write_to_gexfs(G_init, changes, path)

In [26]:
# Not varying n or capacity. Varying algorithms and directedness.
n = 100
capacity = 20

experiments = { 
          0:['binomial', 'binomial', 'undirected', 'undirected_binomial_graph_no_reallocation_n100_c20'], 
          1: ['scale-free', 'scale-free', 'undirected', 'undirected_scalefree_graph_no_reallocation_n100_c20'],
          2: ['binomial', 'binomial', 'directed', 'directed_binomial_graph_no_reallocation_n100_c20'],
          3: ['scale-free', 'scale-free', 'directed', 'directed_scalefree_graph_no_reallocation_n100_c20'],
        #   4:['binomial', 'binomial', 'directed', 'directed_binomial_graph_binomial_reallocation_n100_c20'], 
        #   5: ['binomial', 'scale-free', 'directed', 'directed_binomial_graph_scalefree_reallocation_n100_c20'],
        #   6: ['scale-free', 'binomial', 'directed', 'directed_scalefree_graph_binomial_reallocation_n100_c20'],
        #   7: ['scale-free', 'scale-free', 'directed', 'directed_scalefree_graph_scalefree_reallocation_n100_c20']
          }


for i in range(len(experiments.items())):
    run_experiment(n, capacity, experiments[i])

Experiment, undirected_binomial_graph_no_reallocation_n100_c20
Experiment, undirected_scalefree_graph_no_reallocation_n100_c20
Experiment, directed_binomial_graph_no_reallocation_n100_c20
Experiment, directed_scalefree_graph_no_reallocation_n100_c20
